In [13]:
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyomo.opt import SolverFactory
import os

In [14]:
# data
data = pd.read_excel('inputs.xlsx')

In [15]:
def solve_opt(data, pass_min=0, pass_max=5990, mult_1=1, mult_2=1, tee=False):
    model = pyo.ConcreteModel() # create model
    # vary mult 1 to create pareto front!!!
    t = 96
    k = 100
    D_passenger = (data['Total demand']* mult_1).tolist()
    O_energy = (data['Outage']* mult_2).tolist()
    Riding_price = 0.5
    P_buy = 0.1
    P_sell = 0.4
    alpha = 10
    beta = 7.2
    ch_eff = 0.90
    dch_eff = 1/0.9
    gama = 1.5 # 30km/h
    E_0 = 0.2
    E_min = 0.2
    E_max = 1
    R_bat = 150
    C_bat = 80
    Cycle = 3000
    DoD = 60
    T = 96
    delta_t = 0.25
    
    # sets
    model.T = pyo.RangeSet(t) # set of timesteps
    model.K = pyo.RangeSet(k) # set of SAEVs

    # parameters
    model.D_passenger = pyo.Param(model.T, initialize=lambda model, t: D_passenger[t-1]) # operational voltage of charger n
    model.O_energy = pyo.Param(model.T, initialize=lambda model, t: O_energy[t-1]) # operational voltage of charger n
    
    # binary variables
    model.b = pyo.Var(model.K, model.T, within=pyo.Binary) # binary variable indicating if bus k is serving trip i at time t
    model.x = pyo.Var(model.K, model.T, domain=pyo.Binary) # binary variable indicating if bus k is occupying a charger n at time t to charge
    model.y = pyo.Var(model.K, model.T, domain=pyo.Binary) # binary variable indicating if bus k is occupying a charger n at time t to discharge

    # non-negative variables
    model.e = pyo.Var(model.K, model.T, within=pyo.NonNegativeReals) # energy level of bus k at time t
    model.w_buy = pyo.Var(model.T, within=pyo.NonNegativeReals) # electricity purchased from the grid at time t
    model.w_sell = pyo.Var(model.T, within=pyo.NonNegativeReals) # electricity sold to the grid at time t
    model.w_riding = pyo.Var(model.T, within=pyo.NonNegativeReals) # number of trips performed at time t
    model.d = pyo.Var(model.T, within=pyo.NonNegativeReals) # total degradation cost of the bus k battery at time t

    # constraints
    model.constraints = pyo.ConstraintList()  # Create a set of constraints

    for k in model.K:
        for t in model.T:
            model.constraints.add(model.b[k,t] + model.x[k,t] + model.y[k,t] <=1)

    #for t in model.T:
        model.constraints.add(sum(model.b[k, t] for k in model.K) <= (model.D_passenger[t]))

    for t in model.T:
        model.constraints.add(sum(model.b[k, t] for k in model.K) == model.w_riding[t])
     
    # passengers   
    model.constraints.add(sum(model.w_riding[t] for t in model.T) >= pass_min)
    model.constraints.add(sum(model.w_riding[t] for t in model.T) <= pass_max)


    for k in model.K:
        for t in range(2,T+1):
            model.constraints.add(model.e[k,t] == model.e[k,t-1] + delta_t*ch_eff*alpha*model.x[k,t] - gama*model.b[k,t] - delta_t *dch_eff*beta*model.y[k,t])

    for t in model.T:
        model.constraints.add(delta_t * sum(ch_eff*alpha*model.x[k,t] for k in model.K) == model.w_buy[t])

    for t in model.T:
            model.constraints.add(delta_t * sum(dch_eff*beta*model.y[k,t] for k in model.K) == model.w_sell[t])

    for t in model.T:
        model.constraints.add(delta_t * sum(dch_eff*beta*model.y[k,t] for k in model.K) <= (delta_t * model.O_energy[t]))

    for k in model.K:
        for t in model.T:
            model.constraints.add(model.e[k,t] >= C_bat * E_min)

    for k in model.K:
        for t in model.T:
            model.constraints.add(E_max * C_bat >= model.e[k,t])          

    for k in model.K:
        model.constraints.add(model.e[k,1] == E_0*C_bat)

        for t in model.T:
            model.constraints.add(model.d[t] == ((R_bat)/(Cycle*DoD) * model.w_sell[t]))

    # objective function
    def rule_obj(mod):
        return sum(Riding_price*mod.w_riding[t] for t in mod.T) - sum(P_buy*mod.w_buy[t] for t in mod.T) + sum(P_sell*mod.w_sell[t] for t in mod.T) - sum(mod.d[t] for t in mod.T) 
    model.obj = pyo.Objective(rule=rule_obj, sense=pyo.maximize)
    
    # SOLVER
    opt = pyo.SolverFactory('gurobi')
    opt.options['timelimit'] = 600
    opt.options['mipgap'] = 0.01
    results = opt.solve(model, tee=tee)
    #print(results)
    return model

In [16]:
# Save function
def save_to_excel(model, filename='output.xlsx', Riding_price = 0.5, P_buy = 0.1, P_sell = 0.25, delta_t = 0.25):

    # Create a dictionary to store variable values
    var_values = {}

    # Store values of binary variables
    var_values['b'] = {(k, t): pyo.value(model.b[k, t]) for k in model.K for t in model.T}
    var_values['x'] = {(k, t): pyo.value(model.x[k, t]) for k in model.K for t in model.T}
    var_values['y'] = {(k, t): pyo.value(model.y[k, t]) for k in model.K for t in model.T}

    # Store values of non-negative variables
    var_values['e'] = {(k, t): pyo.value(model.e[k, t]) for k in model.K for t in model.T}
    var_values['w_buy'] = {t: pyo.value(model.w_buy[t]) for t in model.T}
    var_values['w_sell'] = {t: pyo.value(model.w_sell[t]) for t in model.T}
    var_values['w_riding'] = {t: pyo.value(model.w_riding[t]) for t in model.T}
    var_values['d'] = {(t): pyo.value(model.d[t]) for t in model.T}

    # Store objective function values
    total_revenues = pyo.value(sum(Riding_price * model.w_riding[t] for t in model.T) - sum(delta_t*P_buy * model.w_buy[t] for t in model.T) + sum(delta_t*P_sell * model.w_sell[t] for t in model.T) - sum(model.d[t] for t in model.T))
    total_passengers = pyo.value(sum(model.w_riding[t] for t in model.T))
    total_energy_provided = pyo.value(delta_t*sum(model.w_sell[t] for t in model.T))
    total_energy_charged = pyo.value(delta_t*sum(model.w_buy[t] for t in model.T))
    total_degradation = pyo.value(sum(model.d[t] for t in model.T))

    objective_values = {
        'total_revenues': total_revenues,
        'total_passengers': total_passengers,
        'total_energy_provided': total_energy_provided,
        'total_energy_charged': total_energy_charged,
        'total_degradation': total_degradation
    }

    # Convert dictionary to DataFrame and save to Excel
    with pd.ExcelWriter(filename) as writer:
        for var, values in var_values.items():
            df = pd.DataFrame.from_dict(values, orient='index', columns=[var])
            df.to_excel(writer, sheet_name=var)
        
        obj_df = pd.DataFrame.from_dict(objective_values, orient='index', columns=['Value'])
        obj_df.to_excel(writer, sheet_name='Objective_Function_Values')

In [17]:
# Set global font size parameters
plt.rcParams.update({'font.size': 14, 'axes.titlesize': 16, 'axes.labelsize': 14,
                     'xtick.labelsize': 12, 'ytick.labelsize': 12, 'legend.fontsize': 12})

def plot_variables(model, D_passenger, O_energy, t=96, k=100):
    # Convert time steps to hours
    time_steps = range(1, t + 1)
    hours = [step / 4 for step in time_steps]
    bar_width = 0.2  # Adjust the bar width to prevent overlap

    # 1. Percentage of the fleet that is: serving a trip, charging, discharging, or idle
    serving = [sum(pyo.value(model.b[k, t]) for k in model.K) for t in model.T]
    charging = [sum(pyo.value(model.x[k, t]) for k in model.K) for t in model.T]
    discharging = [sum(pyo.value(model.y[k, t]) for k in model.K) for t in model.T]
    idle = [k - s - c - d for s, c, d in zip(serving, charging, discharging)]

    plt.figure(figsize=(15, 5))

    plt.bar(hours, serving, width=bar_width, label='Serving', color='blue', alpha=0.6, edgecolor='black', linewidth=0.5)
    plt.bar(hours, charging, width=bar_width, bottom=serving, label='Charging', color='green', alpha=0.6, edgecolor='black', linewidth=0.5)
    plt.bar(hours, discharging, width=bar_width, bottom=[s + c for s, c in zip(serving, charging)], label='Discharging', color='red', alpha=0.6, edgecolor='black', linewidth=0.5)
    plt.bar(hours, idle, width=bar_width, bottom=[s + c + d for s, c, d in zip(serving, charging, discharging)], label='Idle', color='gray', alpha=0.6, edgecolor='black', linewidth=0.5)
    plt.xlabel('Time [h]')
    plt.ylabel('Number of Vehicles')
    plt.xticks(range(0, 25, 1))  # Set x-axis to show every hour from 0 to 24
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=4)
    plt.tight_layout()
    plt.show()

    # 2. Summation of the energy level of the fleet
    total_energy = [sum(pyo.value(model.e[k, t]) for k in model.K) for t in model.T]
    plt.figure(figsize=(7.5, 5))
    plt.plot(hours, total_energy, label='Total Energy Level')
    plt.xlabel('Time [h]')
    plt.ylabel('Energy [kWh]')
    plt.xticks(range(0, 25, 1))  # Set x-axis to show every hour from 0 to 24
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=4)
    plt.tight_layout()
    plt.show()

    # 3. Chart with only w_buy and w_sell
    plt.figure(figsize=(7.5, 5))
    plt.plot(hours, [pyo.value(model.w_buy[t]) for t in model.T], label='Charging')
    plt.plot(hours, [pyo.value(model.w_sell[t]) for t in model.T], label='Discharging')
    plt.xlabel('Time [h]')
    plt.ylabel('Power [kW]')
    plt.xticks(range(0, 25, 1))  # Set x-axis to show every hour from 0 to 24
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=4)
    plt.tight_layout()
    plt.show()

    # 4. Plot number of vehicles serving a trip and passenger demand
    serving = [(sum(pyo.value(model.b[k, t]) for k in model.K) / k) * 100 for t in model.T]

    fig, ax1 = plt.subplots(figsize=(15, 5))

    color = 'tab:blue'
    ax1.set_xlabel('Time [h]')
    ax1.set_ylabel('Vehicles (%)', color=color)
    ax1.bar(hours, serving, width=bar_width, label='Vehicles Serving', color=color, alpha=0.6, edgecolor='black', linewidth=0.5)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()
    color = 'tab:orange'
    ax2.set_ylabel('Number of Passengers', color=color)
    ax2.plot(hours, D_passenger, label='Passenger Demand', color=color, linestyle='--')
    ax2.tick_params(axis='y', labelcolor=color)

    # Align the zero point for both y-axes
    ax1.set_ylim(bottom=0)
    ax2.set_ylim(bottom=0)
    plt.xticks(range(0, 25, 1))  # Set x-axis to show every hour from 0 to 24
    fig.tight_layout()
    fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0), ncol=2)
    plt.show()

    # 5. Plot relation of vehicles discharging and the number of outage demand
    discharging = [(sum(pyo.value(model.y[k, t]) for k in model.K) / k) * 100 for t in model.T]

    fig, ax1 = plt.subplots(figsize=(15, 5))

    color = 'tab:red'
    ax1.set_xlabel('Time [h]')
    ax1.set_ylabel('Vehicles (%)', color=color)
    ax1.bar(hours, discharging, width=bar_width, label='Vehicles Discharging (%)', color=color, alpha=0.6, edgecolor='black', linewidth=0.5)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()
    color = 'tab:green'
    ax2.set_ylabel('Power Demand [kW]', color=color)
    ax2.plot(hours, O_energy, label='Outage Demand', color=color, linestyle='--')
    ax2.tick_params(axis='y', labelcolor=color)
    
    # Align the zero point for both y-axes
    ax1.set_ylim(bottom=0)
    ax2.set_ylim(bottom=0)

    plt.xticks(range(0, 25, 1))  # Set x-axis to show every hour from 0 to 24
    # Ensure both y-axes are well visible
    fig.tight_layout()
    fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0), ncol=2)
    plt.show()


In [18]:
passengers = [1750,2000,2250,2500,2750,3000,3250,3500,3750,4000,4250,4500,4750,5000,5250,5500,5750]    
for passenger in passengers:
    model = solve_opt(data,mult_2=1, pass_min = passenger, tee=False) # vary pass_min=1,642
    # Create a dynamic filename based on the scenario
    filename = os.path.join('pareto_results', f'output_energy_{passenger}.xlsx')
    # Save results to Excel
    save_to_excel(model, filename=filename)
    print(f"Results for {passenger} saved to {filename}")

Results for 1750 saved to pareto_results/output_energy_1750.xlsx
Results for 2000 saved to pareto_results/output_energy_2000.xlsx
Results for 2250 saved to pareto_results/output_energy_2250.xlsx
Results for 2500 saved to pareto_results/output_energy_2500.xlsx
Results for 2750 saved to pareto_results/output_energy_2750.xlsx
Results for 3000 saved to pareto_results/output_energy_3000.xlsx
Results for 3250 saved to pareto_results/output_energy_3250.xlsx
Results for 3500 saved to pareto_results/output_energy_3500.xlsx
Results for 3750 saved to pareto_results/output_energy_3750.xlsx
Results for 4000 saved to pareto_results/output_energy_4000.xlsx
Results for 4250 saved to pareto_results/output_energy_4250.xlsx
Results for 4500 saved to pareto_results/output_energy_4500.xlsx
Results for 4750 saved to pareto_results/output_energy_4750.xlsx
Results for 5000 saved to pareto_results/output_energy_5000.xlsx
Results for 5250 saved to pareto_results/output_energy_5250.xlsx
Results for 5500 saved to